In [1]:
pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.5/483.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.5 MB/s eta 0:00:00

In [2]:
import pycaret
pycaret.__version__

'3.0.1'

In [20]:

from google.colab import files 
 
uploaded = files.upload()

Saving insurance_pre.csv to insurance_pre.csv


In [22]:
import pandas as pd
data = pd.read_csv("insurance_pre.csv")
data

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [23]:
data = dataset.sample(frac=0.9, random_state=786)
data_unseen = dataset.drop(data.index)

data.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

Data for Modeling: (1204, 7)
Unseen Data For Predictions: (134, 7)


In [24]:

from pycaret.regression import *
exp_reg101 = setup(data = data, target = 'charges', session_id=123,
                  bin_numeric_features = ['age', 'bmi'])

,Description,Value
0,Session id,123
1,Target,charges
2,Target type,Regression
3,Original data shape,"(1204, 7)"
4,Transformed data shape,"(1204, 10)"
5,Transformed train set shape,"(842, 10)"
6,Transformed test set shape,"(362, 10)"
7,Ordinal features,2
8,Numeric features,3
9,Categorical features,3


In [25]:
best = compare_models(sort = 'R2')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,2607.1510,20947726.3142,4507.9854,0.8479,0.4160,0.2981,0.2730
lightgbm,Light Gradient Boosting Machine,2838.3531,22984843.7012,4730.3442,0.8338,0.4684,0.3351,0.3940
rf,Random Forest Regressor,2986.4837,26870902.9154,5117.7555,0.8062,0.4841,0.3614,0.5290
ada,AdaBoost Regressor,4859.5579,32308104.0512,5661.4521,0.7738,0.6892,0.8729,0.2650
et,Extra Trees Regressor,3089.3571,32097084.0187,5605.4220,0.7689,0.5097,0.3762,0.3250
xgboost,Extreme Gradient Boosting,3347.6318,32763617.7000,5661.8540,0.7648,0.5936,0.4116,0.4430
ridge,Ridge Regression,4232.3364,35974316.9250,5969.5745,0.7452,0.6574,0.4468,0.2070
br,Bayesian Ridge,4225.7109,35974528.7922,5969.1352,0.7451,0.6641,0.4458,0.2140
lar,Least Angle Regression,4220.8139,35973100.2993,5968.5553,0.7450,0.6810,0.4450,0.2040
llar,Lasso Least Angle Regression,4219.2407,35972329.4679,5968.5397,0.7450,0.6707,0.4447,0.2040


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

In [26]:
tuned_best = tune_model(best)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,4318.0196,34269348.5383,5854.0028,0.7817,0.6138,0.6997
1,4334.6732,31599879.9299,5621.3770,0.8329,0.5679,0.6469
2,4072.7004,26571691.1703,5154.7736,0.7942,0.5584,0.6113
3,3926.1773,29923985.6606,5470.2820,0.7420,0.5679,0.5677
4,4390.8714,40943392.2177,6398.7024,0.7409,0.5040,0.4530
5,4507.8106,34120726.6430,5841.2949,0.7627,0.6035,0.6722
6,4831.6152,43732834.3955,6613.0806,0.7597,0.5874,0.6386
7,4257.7752,32130499.6394,5668.3772,0.6995,0.6905,0.8154
8,4608.8819,32788596.5874,5726.1328,0.8381,0.6468,0.7852


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [27]:

evaluate_model(tuned_best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [28]:
# Evaluate the tuned model on the test set
predictions = predict_model(tuned_best, data=data_unseen)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Gradient Boosting Regressor,3213.8681,33184226.1533,5760.5752,0.7827,0.4537,0.3165


In [29]:

final_model = finalize_model(tuned_best)

In [30]:
print(final_model)

Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['age', 'bmi', 'children'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=['sex', 'smoker', 'region'],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('ordinal_encoding',
                 TransformerWrapper(include=['sex', 'smoker...
dtype: int64}]))),
                ('onehot_encoding',
                 TransformerWrapper(include=['region'],
                                    transformer=OneHotEncoder(cols=['region'],
                                                              handle_missing='return_nan',
                                                              use_cat_names=True))),
                ('bin_numeric_features',
                 TransformerWrapper(include=['age', 'bmi'],
  

In [31]:
# Save the tuned model
save_model(final_model, 'final_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['age', 'bmi', 'children'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=['sex', 'smoker', 'region'],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('ordinal_encoding',
                  TransformerWrapper(include=['sex', 'smoker...
 dtype: int64}]))),
                 ('onehot_encoding',
                  TransformerWrapper(include=['region'],
                                     transformer=OneHotEncoder(cols=['region'],
                                                               handle_missing='return_nan',
                                                               use_cat_names=True))),
                 ('bin_numeric_features',
                  TransformerWrapper(include=[

In [32]:

# Print the evaluation results
print(predictions)

     age     sex        bmi  children smoker     region       charges  \
0     31  female  25.740000         0     no  southeast   3756.621582   
1     37  female  27.740000         3     no  northwest   7281.505371   
2     56  female  39.820000         0     no  southeast  11090.717773   
3     23    male  23.844999         0     no  northeast   2395.171631   
4     19  female  28.600000         5     no  southwest   4687.796875   
..   ...     ...        ...       ...    ...        ...           ...   
129   45    male  30.360001         0    yes  southeast  62592.875000   
130   43    male  27.799999         0    yes  southwest  37829.722656   
131   33  female  26.695000         0     no  northwest   4571.413086   
132   42  female  40.369999         2    yes  southeast  43896.375000   
133   61  female  29.070000         0    yes  northwest  29141.359375   

     prediction_label  
0         3943.327098  
1         8873.847158  
2        13009.560652  
3         2695.662489  
4  